In [1]:
import torch
import torch.nn as nn
import random

In [2]:
adj_list = [
[1,2,3], 
[0,2,3], 
[0, 1, 3], 
[0, 1, 2], 
[5, 6], 
[4,6], 
[4, 5], 
[1, 3]]
size_vertex = len(adj_list)

In [3]:
w=3  # 单边窗口长度，包括中心词
d=2  # 中间维度
y=200  # 每节点采样次数
t=6  # 每节点采样长度
lr=0.025
v=[0, 1, 2, 3, 4, 5, 6, 7]

In [4]:
class Model(torch.nn.Module) :

    def __init__(self) :
        super(Model, self).__init__()
        self.phi1 = nn.Parameter(torch.rand((size_vertex, d), requires_grad=True))  # 8to2
        self.phi2 = nn.Parameter(torch.rand((d, size_vertex), requires_grad=True))  # 2to8 softmax

    def forward(self, hot) :
        hid = torch.matmul(hot, self.phi1)
        out = torch.matmul(hid, self.phi2)
        return out

In [5]:
def RandomWalk(node, t) :

    walk = [node]

    for i in range(t-1) :
        node = adj_list[node][random.randint(0, len(adj_list[node])-1)]  # 等概率
        walk.append(node)

    return walk

In [6]:
def skip_gram(wvi, w) :

    for j in range(len(wvi)) :

        for k in range( max(0, j-w), min(j+w, len(wvi)) ) :
        # 双重循环，制作语料

            # 中心词的one-hot
            one_hot = torch.zeros(size_vertex)
            one_hot[wvi[j]] = 1

            out = model(one_hot)  # 预测共现词的概率分布
            # print(out)

            loss = torch.log(torch.sum(torch.exp(out))) - out[wvi[k]]  # 做了j*k次loss回传
            loss.backward()  # 回传优化参数

            for param in model.parameters() :
                param.data.sub_(lr*param.grad)
                param.grad.data.zero_()

In [7]:
model = Model()
for i in range(y) :
    random.shuffle(v)  # 原位打乱列表
    for vi in v :
        wvi = RandomWalk(vi, t)
        skip_gram(wvi, w)

In [8]:
print(model.phi1)
print(model.phi2)

Parameter containing:
tensor([[ 1.2468,  0.0530],
        [ 0.9337,  0.8903],
        [ 1.3328,  0.3158],
        [ 0.9518,  0.7068],
        [-1.0665, -1.0001],
        [-1.2288, -0.7669],
        [-0.7969, -1.2464],
        [-0.0708,  1.4122]], requires_grad=True)
Parameter containing:
tensor([[ 2.5237,  2.0535,  2.3813,  2.2814, -2.1476, -2.3450, -1.7781,  0.0599],
        [ 1.1577,  1.9726,  1.3898,  1.8684, -1.3028, -1.2479, -1.4468,  1.9685]],
       requires_grad=True)
